# MPM ALGORITHM FOR MAX FLOW

# IMPORTS

In [10]:
from pyvis.network import Network
from typing import List, Tuple
import math
from collections import Counter, deque, defaultdict
import urllib.request
import time
from pathlib import Path
import shutil

In [11]:
"""
FordFulkeron algorithm for maximum flow problem 
- pluggable augmenting path finding algorithms
- residual graph
- bottleneck capacity
- flow edge class
- depth first search
- Edmonds-Karp algorithm
- capacity scaling algorithm
- dinics algorithm
-- deadend elimination
TODO: May need to merge parallel edges at some point

"""
class FlowEdge:
    def __init__(self, src: int, dst: int, cap: int):
        self.src = src # source node
        self.dst = dst # destination node
        self.cap = cap
        self.flow = 0

    def __repr__(self):
        return f'source node: {self.src}, destination node: {self.dst}, capacity: {self.cap}, flow: {self.flow} ======'

class FordFulkersonMaxFlowV2:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    - residual graph
    - bottleneck capacity
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.flowedges = []
        self.adj_list = {}
        self.delta = 0
        for u, v, cap in edges:
            self.flowedges.append(FlowEdge(u, v, cap))
            if u not in self.adj_list:
                self.adj_list[u] = []
            self.adj_list[u].append(len(self.flowedges) - 1)
            self.flowedges.append(FlowEdge(v, u, 0)) # residual edge
            if v not in self.adj_list:
                self.adj_list[v] = []
            self.adj_list[v].append(len(self.flowedges) - 1)
            self.delta = max(self.delta, cap)
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.vis = [0] * self.size

    def neighborhood(self, node: int) -> List[int]:
        return (i for i in self.adj_list[node])

    def residual_capacity(self, edge: FlowEdge) -> int:
        return edge.cap - edge.flow

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and self.residual_capacity(nei) > 0:
                cur_flow = self.dfs(nei.dst, sink, min(flow, self.residual_capacity(nei)))
                if cur_flow > 0:
                    nei.flow += cur_flow
                    self.flowedges[index ^ 1].flow -= cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            self.parents = [-1] * len(self.flowedges)
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf, -1)])
        self.vis[source] = 1
        while queue:
            node, flow, prev_index = queue.popleft()
            if node == sink:
                break
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.vis[nei.dst] == 0 and self.residual_capacity(nei) > 0:
                    self.vis[nei.dst] = 1
                    self.parents[index] = prev_index
                    queue.append((nei.dst, min(flow, self.residual_capacity(nei)), index))
        if node == sink:
            while prev_index != -1:
                parent_index = self.parents[prev_index]
                self.flowedges[prev_index].flow += flow
                self.flowedges[prev_index^1].flow -= flow # residual edge
                prev_index = parent_index
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and self.residual_capacity(nei) >= self.delta:
                cur_flow = self.capacity_scaling(nei.dst, sink, min(flow, self.residual_capacity(nei)))
                if cur_flow > 0:
                    nei.flow += cur_flow
                    self.flowedges[index ^ 1].flow -= cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.distances[nei.dst] == -1 and self.residual_capacity(nei) > 0:
                    self.distances[nei.dst] = self.distances[node] + 1
                    queue.append(nei.dst)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        while self.ptr[node] < len(self.adj_list[node]):
            index = self.adj_list[node][self.ptr[node]]
            self.ptr[node] += 1
            nei = self.flowedges[index]
            if self.distances[nei.dst] == self.distances[node] + 1 and self.residual_capacity(nei) > 0:
                cur_flow = self.dinics_dfs(nei.dst, sink, min(flow, self.residual_capacity(nei)))
                if cur_flow > 0:
                    nei.flow += cur_flow
                    self.flowedges[index ^ 1].flow -= cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        flowvis = FlowVisualizer()
        flowvis.open(self.edges)
        while self.dinics_bfs(source, sink):
            self.reset()
            self.ptr = [0] * self.size # pointer to the next edge to be processed (optimizes for dead ends)
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                flowvis.update(self.flowedges)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        
        return maxflow
    
    def general_path_cover(self, source: int, sink: int) -> int:
        self.path, self.paths = [], []
        self.vis = [0] * len(self.flowedges)
        for index in self.neighborhood(source):
            fedge = self.flowedges[index]
            if fedge.flow != 1: continue
            self.vis[index] = 1
            self.path.append(source)
            self.path_dfs(self.flowedges[index].dst, sink, source)
            self.path.pop()
        return self.paths

    def path_dfs(self, node: int, sink: int, parent: int) -> None:
        print('node', node, 'path', self.path)
        if node == sink:
            self.paths.append([i + 1 for i in self.path + [node]])
            return
        for index in self.neighborhood(node):
            fedge = self.flowedges[index]
            if fedge.dst == parent: continue
            if self.vis[index]: continue
            self.vis[index] = 1
            print('node', node, 'fedge', fedge)
            if fedge.flow == 1:
                self.path.append(node)
                self.path_dfs(fedge.dst, sink, node)
                self.path.pop()
                return


In [12]:
class FlowVisualizer:
    def open(self, edges, directory = 'flow_network_vis'):
        self.net = Network('1024px', '600px', notebook = True, directed = True)
        self.net.toggle_physics(False)
        self.net.set_edge_smooth('dynamic') # deals with parallel edges?
        # self.net.show_buttons(filter_ = ['physics', 'nodes', 'edges'])
        options = """
const options = {
  "nodes": {
    "borderWidth": 4,
    "borderWidthSelected": null,
    "opacity": null,
    "size": null
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "springConstant": 0.005,
      "damping": 1
    },
    "minVelocity": 0.75
  }
}
        """
        self.net.set_options(options)
        self.cnt = 0
        shutil.rmtree(directory, ignore_errors = True)
        Path(directory).mkdir(parents = True, exist_ok = True)
        self.directory = directory
        self.build(edges)

    def node_adder(self, node: int):
        if node not in self.nodes:
            self.net.add_node(node, title = f'{node}', size = 20, label = f'{node}')
            self.nodes.add(node)

    def build(self, edges):
        self.nodes = set()
        self.edges_index = {edge: i for i, edge in enumerate(edges)} # edge will be a tuple of (src, dst, cap)
        self.edges_data = {i: {'flow': 0, 'capacity': 0} for i in range(len(edges))} # edge data is interested in flow and capacity
        for i, (src, dst, cap) in enumerate(edges):
            self.node_adder(src)
            self.node_adder(dst)
            self.net.add_edge(src, dst)
            self.edges_data[self.edges_index[(src, dst, cap)]]['capacity'] = cap
            self.net.edges[i]['label'] = '/\n'.join(f'{key}: {value}' for key, value in self.edges_data[i].items())
        self.net.show(f'{self.directory}/augmenting_path_{self.cnt:04}.html')

    """
    This takes in flow edge class to update the flow network
    """
    def update(self, flowedges):
        self.cnt += 1
        for flowedge in flowedges:
            src, dst, cap, flow = flowedge.src, flowedge.dst, flowedge.cap, flowedge.flow
            edge_index = self.edges_index.get((src, dst, cap), None)
            if edge_index is not None:
                self.edges_data[edge_index]['flow'] = flow
                self.net.edges[edge_index]['label'] = '/\n'.join(f'{key}: {value}' for key, value in self.edges_data[edge_index].items())
        self.net.show(f'{self.directory}/augmenting_path_{self.cnt:04}.html')
    
    """
    Does nothing, at this point do not need to close it.  But I just want to use this functions, and doesn't make sense to not have open without close
    """
    def close(self):
        passs
    


In [13]:
url = 'https://cses.fi/file/d622b21fd0a6f921bf07317d5685afafb9bbd7053e128c17f0307f5e93ba22b8/1/1/'
data = urllib.request.urlopen(url)
for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
    if j == 0:
        n, m = map(int, line.split())
        edges = []
    else:
        u, v = map(int, line.split())
        edges.append((u - 1, v - 1, 1))
# calls dinics function, now I need for each iteration to have it visualize the network
# need to conver the dinics function to an iterable or something, so that I can stop at each one
# and display the results
mf_dinics = FordFulkersonMaxFlowV2(n, edges).main_dinics(0, n - 1)
print(mf_dinics)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
2
